# Getting Started

## Creating DataFrames

In [4]:
# set filename(in hdfs)
userJSONFileName = "hdfs://hadoop:9000/data/ml-1m/users.json"
# read json file
# - spark           -> pyspark.sql.SparkSession
# - spark.read      -> pyspark.sql.DataFrameReader
# - spark.read.json -> pyspark.sql.DataFrame
df = spark.read.json(userJSONFileName)
# Displays the content of the DataFrame to stdout
df.show()

+---+------+----------+------+-------+
|age|gender|occupation|userID|zipcode|
+---+------+----------+------+-------+
|  1|     F|        10|     1|  48067|
| 56|     M|        16|     2|  70072|
| 25|     M|        15|     3|  55117|
| 45|     M|         7|     4|  02460|
| 25|     M|        20|     5|  55455|
| 50|     F|         9|     6|  55117|
| 35|     M|         1|     7|  06810|
| 25|     M|        12|     8|  11413|
| 25|     M|        17|     9|  61614|
| 35|     F|         1|    10|  95370|
| 25|     F|         1|    11|  04093|
| 25|     M|        12|    12|  32793|
| 45|     M|         1|    13|  93304|
| 35|     M|         0|    14|  60126|
| 25|     M|         7|    15|  22903|
| 35|     F|         0|    16|  20670|
| 50|     M|         1|    17|  95350|
| 18|     F|         3|    18|  95825|
|  1|     M|        10|    19|  48073|
| 25|     M|        14|    20|  55113|
+---+------+----------+------+-------+
only showing top 20 rows



## Untyped Dataset Operations (aka DataFrame Operations)

In [2]:
# Print the schema in a tree format
df.printSchema()

root
 |-- age: long (nullable = true)
 |-- gender: string (nullable = true)
 |-- occupation: string (nullable = true)
 |-- userID: long (nullable = true)
 |-- zipcode: string (nullable = true)



## 光环练习作业

- 读取文件 `ml-1m/ratings.data` 中的内容，并将其转换为 **DataFrame/Dataset**

> 这道题的目的是考量从 RDD 转换为 DataFrame 的能力
> `ml-1m/ratings.data` 文件的说明，请查看 [Ratings File Description](#RATINGS-FILE-DESCRIPTION)

**思路**：为了将 RDD 转换为 DataFrame spark 提供了两种方式
- Inferring the Schema Using Reflection
- Programmatically Specifying the Schema

In [28]:
# 1st way: Inferring the Schema Using Reflection
from pyspark.sql import Row

# Load a text file and convert each line to a Row
lines = sc.textFile("hdfs://hadoop:9000/data/ml-1m/ratings.dat")
parts = lines.map(lambda l: l.split("::"))
rating = parts.map(lambda p: Row(userID=int(p[0]), movieID=int(p[1]), rating=int(p[2]), timestamp=int(p[3])))

# Infer the schame, and register the DataFrame as a table.
dfRating = spark.createDataFrame(rating)
dfRating.createOrReplaceTempView("rating")

# SQL can be run over DataFrames that have been registered as table.
results = spark.sql("select * from rating limit 3")
results.collect()

[Row(movieID=1193, rating=5, timestamp=978300760, userID=1),
 Row(movieID=661, rating=3, timestamp=978302109, userID=1),
 Row(movieID=914, rating=3, timestamp=978301968, userID=1)]

In [29]:
results.show()

+-------+------+---------+------+
|movieID|rating|timestamp|userID|
+-------+------+---------+------+
|   1193|     5|978300760|     1|
|    661|     3|978302109|     1|
|    914|     3|978301968|     1|
+-------+------+---------+------+



In [35]:
# 2nd way: Programmatically Specifying the Schema
# Import data types
from pyspark.sql.types import *

# Use above parts
rating = parts.map(lambda p: (int(p[0]), int(p[1]), int(p[2]), int(p[3])))

# The schema is encoded in a string
schemaString = "userID moiveID rating timestamp"

fieldsRating = [StructField(fieldName, IntegerType(), True) for fieldName in schemaString.split()]
schemaRating = StructType(fieldsRating)

# Apply the schema to the RDD
dfRating = spark.createDataFrame(rating, schemaRating)

# Create a temporary view using the DataFrame
dfRating.createOrReplaceTempView("rating")

# SQL can be run over DataFrames that have been registered as a table.
results = spark.sql("select * from rating limit 3")

results.show()

+------+-------+------+---------+
|userID|moiveID|rating|timestamp|
+------+-------+------+---------+
|     1|   1193|     5|978300760|
|     1|    661|     3|978302109|
|     1|    914|     3|978301968|
+------+-------+------+---------+



- 将上面转换的 dataframe 以 Parquet 格式存储到 /tmp/ratings.parquet

SUMMARY
================================================================================

These files contain 1,000,209 anonymous ratings of approximately 3,900 movies 
made by 6,040 MovieLens users who joined MovieLens in 2000.

USAGE LICENSE
================================================================================

Neither the University of Minnesota nor any of the researchers
involved can guarantee the correctness of the data, its suitability
for any particular purpose, or the validity of results based on the
use of the data set.  The data set may be used for any research
purposes under the following conditions:

     * The user may not state or imply any endorsement from the
       University of Minnesota or the GroupLens Research Group.

     * The user must acknowledge the use of the data set in
       publications resulting from the use of the data set
       (see below for citation information).

     * The user may not redistribute the data without separate
       permission.

     * The user may not use this information for any commercial or
       revenue-bearing purposes without first obtaining permission
       from a faculty member of the GroupLens Research Project at the
       University of Minnesota.

If you have any further questions or comments, please contact GroupLens
<grouplens-info@cs.umn.edu>. 

CITATION
================================================================================

To acknowledge use of the dataset in publications, please cite the following
paper:

F. Maxwell Harper and Joseph A. Konstan. 2015. The MovieLens Datasets: History
and Context. ACM Transactions on Interactive Intelligent Systems (TiiS) 5, 4,
Article 19 (December 2015), 19 pages. DOI=http://dx.doi.org/10.1145/2827872


ACKNOWLEDGEMENTS
================================================================================

Thanks to Shyong Lam and Jon Herlocker for cleaning up and generating the data
set.

FURTHER INFORMATION ABOUT THE GROUPLENS RESEARCH PROJECT
================================================================================

The GroupLens Research Project is a research group in the Department of 
Computer Science and Engineering at the University of Minnesota. Members of 
the GroupLens Research Project are involved in many research projects related 
to the fields of information filtering, collaborative filtering, and 
recommender systems. The project is lead by professors John Riedl and Joseph 
Konstan. The project began to explore automated collaborative filtering in 
1992, but is most well known for its world wide trial of an automated 
collaborative filtering system for Usenet news in 1996. Since then the project 
has expanded its scope to research overall information filtering solutions, 
integrating in content-based methods as well as improving current collaborative 
filtering technology.

Further information on the GroupLens Research project, including research 
publications, can be found at the following web site:
        
        http://www.grouplens.org/

GroupLens Research currently operates a movie recommender based on 
collaborative filtering:

        http://www.movielens.org/

RATINGS FILE DESCRIPTION
================================================================================

All ratings are contained in the file "ratings.dat" and are in the
following format:

UserID::MovieID::Rating::Timestamp

- UserIDs range between 1 and 6040 
- MovieIDs range between 1 and 3952
- Ratings are made on a 5-star scale (whole-star ratings only)
- Timestamp is represented in seconds since the epoch as returned by time(2)
- Each user has at least 20 ratings

USERS FILE DESCRIPTION
================================================================================

User information is in the file "users.dat" and is in the following
format:

UserID::Gender::Age::Occupation::Zip-code

All demographic information is provided voluntarily by the users and is
not checked for accuracy.  Only users who have provided some demographic
information are included in this data set.

- Gender is denoted by a "M" for male and "F" for female
- Age is chosen from the following ranges:

	*  1:  "Under 18"
	* 18:  "18-24"
	* 25:  "25-34"
	* 35:  "35-44"
	* 45:  "45-49"
	* 50:  "50-55"
	* 56:  "56+"

- Occupation is chosen from the following choices:

	*  0:  "other" or not specified
	*  1:  "academic/educator"
	*  2:  "artist"
	*  3:  "clerical/admin"
	*  4:  "college/grad student"
	*  5:  "customer service"
	*  6:  "doctor/health care"
	*  7:  "executive/managerial"
	*  8:  "farmer"
	*  9:  "homemaker"
	* 10:  "K-12 student"
	* 11:  "lawyer"
	* 12:  "programmer"
	* 13:  "retired"
	* 14:  "sales/marketing"
	* 15:  "scientist"
	* 16:  "self-employed"
	* 17:  "technician/engineer"
	* 18:  "tradesman/craftsman"
	* 19:  "unemployed"
	* 20:  "writer"

MOVIES FILE DESCRIPTION
================================================================================

Movie information is in the file "movies.dat" and is in the following
format:

MovieID::Title::Genres

- Titles are identical to titles provided by the IMDB (including
year of release)
- Genres are pipe-separated and are selected from the following genres:

	* Action
	* Adventure
	* Animation
	* Children's
	* Comedy
	* Crime
	* Documentary
	* Drama
	* Fantasy
	* Film-Noir
	* Horror
	* Musical
	* Mystery
	* Romance
	* Sci-Fi
	* Thriller
	* War
	* Western

- Some MovieIDs do not correspond to a movie due to accidental duplicate
entries and/or test entries
- Movies are mostly entered by hand, so errors and inconsistencies may exist
